In [1]:
import psycopg2
from os import listdir

In [2]:
DB = 'test'
USER = 'breno'
PORT = 5433  # typically 5432

In [4]:
def startup(database, user, port):
    conn = psycopg2.connect(database=database, user=user, port=port)
    cur = conn.cursor()
    return (conn, cur)

(conn, cur) = startup(DB, USER, PORT)

Next we create the tables according to the formats
found in the CSV files.

In [6]:
def create_accounting_table(conn, cur):
    create_accounting_table_query = (
    '''
    CREATE TABLE accounting (
        data DATE,
        reg_ans INT,
        cd_conta_contabil INT,
        descricao VARCHAR(255),
        vl_saldo_final FLOAT
        );
    '''
    )
    cur.execute('set datestyle to iso;')
    cur.execute(create_accounting_table_query)
    conn.commit()
    
def create_companies_table(conn, cur):
    create_companies_table_query = (
    '''
    CREATE TABLE companies (
        reg_ans INT,
        cnpj VARCHAR(15),
        razao_social VARCHAR(255),
        nome_fantasia VARCHAR(255),
        modalidade VARCHAR(255),
        logradouro VARCHAR(255),
        numero VARCHAR(255),
        complemento VARCHAR(255),
        bairro VARCHAR(255),
        cidade VARCHAR(255),
        uf VARCHAR(255),
        cep VARCHAR(255),
        ddd VARCHAR(255),
        telefone VARCHAR(255),
        fax VARCHAR(255),
        email VARCHAR(255),
        representante VARCHAR(255),
        cargo_representante VARCHAR(255),
        data_registro_ans date
        );
    '''
    )
    cur.execute('set datestyle to dmy;')
    cur.execute(create_companies_table_query)
    conn.commit()


def create_tables(conn, cur,):
    create_accounting_table(conn, cur)
    create_companies_table(conn, cur)

In [ ]:
# only run this if the tables have not been created already
create_tables(conn_cur)

We now need to read the provided CSVs into the tables.

In [7]:
def load_csvs(conn, cur, dir, companies_csv):
    filenames = [dir + filename for filename in listdir(dir)]
    copy_csv_query = (lambda table, filename:
    '''COPY {}
    FROM \'{}\'
    DELIMITER \';\'
    ENCODING \'latin1\'
    CSV HEADER;
    '''.format(table, filename)
    )

    cur.execute(copy_csv_query('companies', companies_csv))

    for filename in filenames:
        cur.execute(copy_csv_query('accounting', filename))
    conn.commit()

In [ ]:
# only run this if data has not been loaded
# check the filenames
dir = '/Users/breno/Documents/intuitive-care/test3/'
companies_csv = dir + 'Relatorio_cadop-2.csv'
load_csvs(conn, cur, dir + 'processed_data', companies_csv)

In [8]:
def fetch_results(description, start_date, end_date):
    query = (
    '''
    select reg_ans,
    sum (vl_saldo_final)
    from accounting
    where descricao=\'{}\'
    and data >= \'{}\'
    and data <  \'{}\'
    group by reg_ans
    order by sum (vl_saldo_final) desc;
    '''.format(description, start_date, end_date)
    )

    cur.execute(query)
    results = cur.fetchall()
    return results

In [13]:
def fetch_companies(results):
    tuples = []
    for (code, value) in results:
        query = (
            'select razao_social from companies where reg_ans={};'
            .format(code)
        )
        cur.execute(query)
        fetch = cur.fetchone()
        name = fetch[0] if fetch else fetch
        tuples.append((name, code, value))
    return tuples

In [15]:
# First query: over the last year
description = 'EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS  DE ASSISTÊNCIA A SAÚDE MEDICO HOSPITALAR '
start_date = '2020-01-01'
end_date = '2021-01-01'
results = fetch_results(description, start_date, end_date)[:10]
companies = fetch_companies(results)
companies

[('BRADESCO SAÚDE S.A.', 5711, 29238637561.909996),
 ('AMIL ASSISTÊNCIA MÉDICA INTERNACIONAL S.A.', 326305, 22239265407.1),
 ('SUL AMERICA COMPANHIA DE SEGURO SAÚDE', 6246, 19134861006.79),
 ('NOTRE DAME INTERMÉDICA SAÚDE S.A.', 359017, 8717609226.0),
 ('CAIXA DE ASSISTÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL',
  346659,
  6701591952.42),
 ('CENTRAL NACIONAL UNIMED - COOPERATIVA CENTRAL', 339679, 5532110614.25),
 ('UNIMED-RIO COOPERATIVA DE TRABALHO MEDICO DO RIO DE JANEIRO',
  393321,
  5449561080.42),
 ('HAPVIDA ASSISTENCIA MEDICA LTDA', 368253, 5296308468.6),
 ('UNIMED BELO HORIZONTE COOPERATIVA DE TRABALHO MÉDICO',
  343889,
  4305426998.96),
 ('GEAP AUTOGESTÃO EM SAÚDE', 323080, 4262564672.2299995)]

In [16]:
# Second query: over the last trimester
description = 'EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS  DE ASSISTÊNCIA A SAÚDE MEDICO HOSPITALAR '
start_date = '2020-07-01'
end_date = '2021-10-01'
results = fetch_results(description, start_date, end_date)[:10]
companies = fetch_companies(results)
companies

[('BRADESCO SAÚDE S.A.', 5711, 14689350312.67),
 ('AMIL ASSISTÊNCIA MÉDICA INTERNACIONAL S.A.', 326305, 10928531414.16),
 ('SUL AMERICA COMPANHIA DE SEGURO SAÚDE', 6246, 9498661307.980001),
 ('NOTRE DAME INTERMÉDICA SAÚDE S.A.', 359017, 4402526741.07),
 ('CAIXA DE ASSISTÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL',
  346659,
  3337556927.0299997),
 ('CENTRAL NACIONAL UNIMED - COOPERATIVA CENTRAL', 339679, 2770465180.87),
 ('UNIMED-RIO COOPERATIVA DE TRABALHO MEDICO DO RIO DE JANEIRO',
  393321,
  2754253192.49),
 ('HAPVIDA ASSISTENCIA MEDICA LTDA', 368253, 2629132597.67),
 ('UNIMED BELO HORIZONTE COOPERATIVA DE TRABALHO MÉDICO',
  343889,
  2211589474.53),
 ('GEAP AUTOGESTÃO EM SAÚDE', 323080, 2062436242.8999999)]